## Imports

In [ ]:
#pip install geopandas

In [ ]:
#pip install fuzzywuzzy

In [53]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point
import argparse
from fuzzywuzzy import process
from dotenv import dotenv_values

C:\Users\carlo\miniconda3\envs\jupyter_env\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Function argparser
def argument_parser():
    # Create ArgumentParser with the app description
    parser = argparse.ArgumentParser(description = 'This app find the BiciMAD/BiciPARK station closest to a set of public\
    schools')
    # Create message to help to the users
    help_message = 'You have two options:\
    \n(1) str="All": to get the table for every "Place of interest" included in the dataset (or a set of them).\
    \n(2) str=school_name: to get the table for a specific "public school" imputed by the user.'  
    # Use '-p' as a flag to select opcion 1 or 2
    parser.add_argument('-p', '--parameter', help=help_message, type=str)
    # Obtain argument
    args = parser.parse_args()
    return args

## Functions

#### Cleaning functions

In [3]:
# FUNCTION 'transform_df'
def transform_df(df):
    # Function to transform and clean the dataframe import from csv.
    
    # Extract the column, delete '[' and ']'. Split the string using ',' and convert to float. Store this data in two
    # columns: longitude and latitude. Add this two columns to the original dataframe
    temp_df = df['geometry.coordinates'].str.strip('[]').str.split(',', expand=True).astype('float64')
    temp_df.columns = ['longitude', 'latitude']
    df= pd.concat([df,temp_df],axis=1)
    
    # Delete column 'geometry.coordinates' and 'Unnamed: 0' columns
    df = df.drop(['Unnamed: 0', 'geometry.coordinates'], axis=1)
    
    # Change the name of 'geometry.type' column becase include '.' in the name, and it could be a potential error
    df = df.rename(columns={'geometry.type':'geometry_type'})
    
    # In case the column names were e.g. 'stationId', extract each column name, if includes 'station', repleace that for 
    # ' '. And change the string to lowercase
    columns = df.columns.tolist()
    new_column_names = [column_name.replace('station', '').lower() for column_name in columns]
    df.columns = new_column_names
    
    return df


# FUNCTION 'extract_dict2df'
def extract_dict2df(df):
    # Function to extract the dictionaries that are included in the cells of some of the columns. Create a new column for each 
    # key and store in it the corresponding values. Finally delete the original columns where the dictionaries are located.
    # With this function it is possible to extract the dictionaries that are in different columns, regardless of the number of 
    # columns that have dictionaries or the number of items in each of them.
    
    column_names = df.columns.values   # Store the column names in a list called 'column_names'

    # In this loop, iterate over the columns of the DataFrame
    for col_name in column_names:
        # Check if the first cell type is a dictionary and, in this case, check if it includes more than 1 items
        if isinstance(df.at[0, col_name], dict) and len(df.at[0, col_name])>1:
            # Extract the keys from the first dictionary found. Use '.at' to get a single value from the DataFrame.
            keys = list(df.at[0, col_name].keys())
 
            # Iterar sobre las claves y agregar nuevas columnas al DataFrame
            for key in keys:
                new_col_name = f"{col_name}_{key}"  # Nombre de la nueva columna
                df[new_col_name] = df[col_name].apply(lambda x: x.get(key))

            # Delete the previous column with the dictionaries inside each cell
            df = df.drop(columns=[col_name])
        
    return df

#### Geo-calculation functions

In [4]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair points in degrees 
    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

## Acquisition and wrangling: import, clean and prepare data

#### Import and clean bicimap.csv

In [5]:
# Read the csv and store the data in a dataframe
bicimad_df = pd.read_csv("../data/raw/bicimad_stations.csv", sep='\t')
# Clean and transform the dataframe
bicimad_df = transform_df(bicimad_df)
# Remove the number (e.g. '1a - , 1b - ....') from the name of each bicimap station
bicimad_df['name'] = bicimad_df['name'].apply(lambda row: row.split(' - ')[1])
bicimad_df.head()

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry_type,longitude,latitude
0,1,Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,-3.701834,40.417214
1,2,Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,-3.701603,40.417313
2,3,Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,-3.705842,40.420589
3,4,Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,-3.706917,40.430294
4,5,Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,-3.702587,40.428552


In [6]:
bicimad_df.to_csv(f"../data/processed/bicimad.csv", index=False)

#### Import and clean bicipark.csv

In [7]:
# Read the csv and store the data in a dataframe
bicipark_df = pd.read_csv("../data/raw/bicipark_stations.csv", sep=';')
# Clean and transform the dataframe
bicipark_df = transform_df(bicipark_df)
# Remove the string 'bicipark ' from the name of each bicipark station
bicipark_df['name'] = bicipark_df['name'].apply(lambda row: row.split('Bicipark ')[1])
bicipark_df.head()

,id,name,address,city,zip_code,total_places,free_places,enabled,reserved_places,geometry_type,longitude,latitude
0,74,Fuente de la Mora,Estación de Fuente de la Mora,Madrid,28050,10,9,1,0,Point,-3.663068,40.484684
1,22,Orense,"Avenida General Perón, 27",Madrid,28020,15,9,1,1,Point,-3.693897,40.452973
2,49,Montalban,"Calle de Montalbán, 5",Madrid,28014,16,15,1,0,Point,-3.690786,40.418146
3,12,Almagro,"Calle Almagro, 11",Madrid,28010,16,12,1,0,Point,-3.692879,40.430361
4,9,Recoletos,"Paseo de Recoletos, 2",Madrid,28001,10,3,0,0,Point,-3.692245,40.420354


In [ ]:
bicipark_df.to_csv(f"../data/processed/bicipark.csv", index=False)

#### Fix and prepare biciMAD and BiciPark dataframes

In [8]:
# Create a new dataframe with columns that these will be use in the final app. 
# Extract some columns from bicimad.df in a new dataframe
new_column_names = {'name': 'station_name', 'address': 'station_location', 'latitude': 'latitude', 'longitude': 'longitude'}
bicimad_stations_df = bicimad_df[list(new_column_names.keys())].rename(columns=new_column_names)
bicimad_stations_df['station_type'] = 'BiciMAD'
bicimad_stations_df.head()

,station_name,station_location,latitude,longitude,station_type
0,Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,BiciMAD
1,Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603,BiciMAD
2,Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842,BiciMAD
3,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917,BiciMAD
4,Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587,BiciMAD


In [9]:
# Extract some columns from bicipark.df in a new dataframe
new_column_names = {'name': 'station_name', 'address': 'station_location', 'latitude': 'latitude', 'longitude': 'longitude'}
bicipark_stations_df = bicipark_df[list(new_column_names.keys())].rename(columns=new_column_names)
bicipark_stations_df['station_type'] = 'BiciPARK'
bicipark_stations_df.head()

,station_name,station_location,latitude,longitude,station_type
0,Fuente de la Mora,Estación de Fuente de la Mora,40.484684,-3.663068,BiciPARK
1,Orense,"Avenida General Perón, 27",40.452973,-3.693897,BiciPARK
2,Montalban,"Calle de Montalbán, 5",40.418146,-3.690786,BiciPARK
3,Almagro,"Calle Almagro, 11",40.430361,-3.692879,BiciPARK
4,Recoletos,"Paseo de Recoletos, 2",40.420354,-3.692245,BiciPARK


In [10]:
stations_df = pd.concat([bicimad_stations_df, bicipark_stations_df])
stations_df

,station_name,station_location,latitude,longitude,station_type
0,Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,BiciMAD
1,Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603,BiciMAD
2,Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842,BiciMAD
3,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917,BiciMAD
4,Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587,BiciMAD
...,...,...,...,...,...
6,Salamanca,Pl. del Marqués de Salamanca,40.430315,-3.679172,BiciPARK
7,Villa de Paris,"Calle de Orellana, 23, 21",40.425608,-3.693698,BiciPARK
8,Pitis,"Calle Pau Arroyo del Fresno, 181",40.495942,-3.724914,BiciPARK
9,Aviación,Parking Aviación Española,40.383492,-3.783695,BiciPARK


#### Import and clean json

In [11]:
url = 'https://datos.madrid.es/egob/catalogo/202311-0-colegios-publicos.json'
response = requests.get(url)

In [12]:
# Obtain json data
json_data = response.json()

# The dictionary has two keys: '@context' and '@graph'. And the interesting data are in the value of the second key where
# other dictionaries are included. Extract both keys in a list called 'keys' -> json_data["@graph"] = json_data[keys[1]]
keys=list(json_data.keys())

# Create the dataframe with the data stored in '@graph'. This way, if the name of the dictionary change, it will still work.
public_schools_df = pd.DataFrame(json_data[keys[1]])

# Use the 'extract_dict2df' function to extract the diccionaries included in some columns and create new columns with them.
public_schools_df = extract_dict2df(public_schools_df) 
#public_schoolss_df.head()      
        

In [13]:
# Extract some columns from bicipark.df in a new dataframe
new_column_names = {'title': 'school_name', 'address_street-address': 'school_location', 
                    'location_latitude': 'latitude', 'location_longitude': 'longitude'}
schools_df = public_schools_df[list(new_column_names.keys())].rename(columns=new_column_names)
schools_df['place_type'] = 'Colegios públicos'
schools_df.head()

,school_name,school_location,latitude,longitude,place_type
0,Colegio Público Adolfo Suárez,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,Colegios públicos
1,Colegio Público Agustina Díez,CALLE VESUBIO 5,40.388191,-3.647237,Colegios públicos
2,Colegio Público Agustín Rodriguez Sahagún,CALLE LA GRANJA DE SAN ILDEFONSO 17,40.368040,-3.622168,Colegios públicos
3,Colegio Público Alberto Alcocer,AVENIDA CANILLEJAS A VICALVARO 56,40.433338,-3.612052,Colegios públicos
4,Colegio Público Alcalde de Móstoles,CALLE ALDEANUEVA DE LA VERA 9,40.379837,-3.776448,Colegios públicos


## Analysis: calculations

#### Geo-calculation

In [ ]:
%%time

# Use a dataframe fragment to test the code
schools_test = schools_df#.iloc[0:2]
bicimad_test = stations_df#.iloc[0:10]
# Filter the dataframe and extract only the rows related to bicimad
#bicimad_test = stations_df.loc[stations_df['station_type'] == 'BiciMAD'].iloc[0:10]

# Merge public schools and bicimad/bicipark dataframe. Before the merge, create a new column called 'key' which value will
# be '1'. Merge using this column and use drop to remove this column in the merged dataframe. This new dataframe will be 
# a dataframe in which 
merge_df = pd.merge(schools_test.assign(key=1), bicimad_test.assign(key=1), on='key').drop('key', axis=1)
# Obtain the distance
merge_df['distance'] = merge_df.apply(lambda row: distance_meters(row['latitude_x'], row['longitude_x'],
                                                             row['latitude_y'], row['longitude_y']), axis=1)
merge_df.head()

In [ ]:
# Store the dataframe in '.csv' and this way I won't have to wait for the distance calculations to be performed if I want 
# to work with the dataframe
merge_df.to_csv("../data/processed/distance_calculated.csv", index=False)


In [14]:
# Import the data to work with them
merge_df = pd.read_csv("../data/processed/distance_calculated.csv", sep=',')

In [15]:
# Short the dataframe for each school from minimum to maximum of the distance from each station to the school that 
# corresponds to it. Reset index and remove the new column index created
merge_short_df = merge_df.sort_values(by=['school_name', 'distance']).reset_index().drop('index', axis=1)
# To extract only the biciMAD items, it's neccesary to apply a filter
bicimad_filter = merge_short_df['station_type'] == 'BiciMAD'
# Obtain the resulting dataframe with the school and bicimad station with minimum distance. As the dataframe is already 
# sorted, with the distance values from smallest to largest, only the first value for each school needs to be extracted. 
# To do this, it's neccesary to apply the filter calculated above.
minimum_df = merge_short_df[bicimad_filter].groupby('school_name').head(1)
minimum_df

,school_name,school_location,latitude_x,longitude_x,place_type,station_name,station_location,latitude_y,longitude_y,station_type,distance
2,Colegio Público Adolfo Suárez,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,Colegios públicos,Manuel Caldeiro,Paseo de la Castellana nº 298,40.479380,-3.685320,BiciMAD,3796.394240
275,Colegio Público Agustina Díez,CALLE VESUBIO 5,40.388191,-3.647237,Colegios públicos,Sierra Toledana,Camino de Valderribas nº 94,40.399015,-3.660777,BiciMAD,1932.123857
550,Colegio Público Agustín Rodriguez Sahagún,CALLE LA GRANJA DE SAN ILDEFONSO 17,40.368040,-3.622168,Colegios públicos,Pavones,Calle Hacienda de Pavones nº 342,40.400368,-3.634587,BiciMAD,3856.153931
825,Colegio Público Alberto Alcocer,AVENIDA CANILLEJAS A VICALVARO 56,40.433338,-3.612052,Colegios públicos,Gutierre de Cetina,Calle Gutierre de Cetina nº 77,40.429706,-3.640994,BiciMAD,3253.471117
1101,Colegio Público Alcalde de Móstoles,CALLE ALDEANUEVA DE LA VERA 9,40.379837,-3.776448,Colegios públicos,Caramuel,Calle Caramuel nº 51,40.409136,-3.728534,BiciMAD,6261.761115
...,...,...,...,...,...,...,...,...,...,...,...
67102,Colegio Público Virgen del Cortijo,AVENIDA MANOTERAS 63,40.487254,-3.660022,Colegios públicos,Manuel Caldeiro,Paseo de la Castellana nº 298,40.479380,-3.685320,BiciMAD,2954.937898
67375,Colegio Público Vázquez de Mella,CALLE BAILEN 18,40.412095,-3.713999,Colegios públicos,Plaza de San Francisco,Plaza de San Francisco nº 5,40.410800,-3.714113,BiciMAD,144.770788
67650,Colegio Público Winston Churchill,AVENIDA GRAN VIA DEL ESTE 5,40.402966,-3.595314,Colegios públicos,Pavones,Calle Hacienda de Pavones nº 342,40.400368,-3.634587,BiciMAD,4390.107731
67925,Colegio Público de Prácticas Asunción Rincón,AVENIDA FILIPINAS 5,40.439916,-3.710968,Colegios públicos,Parque Santander,Calle Santander esquina con Avenida de Filipinas,40.441139,-3.708083,BiciMAD,349.376901


In [43]:
# Change column names and select the desired columns to adapt the result to the objective
# Create a dictionary with the old and new column names
new_columns_names = {'school_name': 'Place of interest',
                     'place_type': 'Type of place',
                     'school_location': 'Place address',
                     'station_name': 'BiciMAD station',
                     'station_location': 'Station location'}
# Extract the interested columns and rename them.
result_df = minimum_df[list(new_columns_names.keys())].rename(columns=new_columns_names).reset_index(drop=True)
result_df

,Place of interest,Type of place,Place address,BiciMAD station,Station location
0,Colegio Público Adolfo Suárez,Colegios públicos,CALLE INFANTA CATALINA MICAELA 31,Manuel Caldeiro,Paseo de la Castellana nº 298
1,Colegio Público Agustina Díez,Colegios públicos,CALLE VESUBIO 5,Sierra Toledana,Camino de Valderribas nº 94
2,Colegio Público Agustín Rodriguez Sahagún,Colegios públicos,CALLE LA GRANJA DE SAN ILDEFONSO 17,Pavones,Calle Hacienda de Pavones nº 342
3,Colegio Público Alberto Alcocer,Colegios públicos,AVENIDA CANILLEJAS A VICALVARO 56,Gutierre de Cetina,Calle Gutierre de Cetina nº 77
4,Colegio Público Alcalde de Móstoles,Colegios públicos,CALLE ALDEANUEVA DE LA VERA 9,Caramuel,Calle Caramuel nº 51
...,...,...,...,...,...
244,Colegio Público Virgen del Cortijo,Colegios públicos,AVENIDA MANOTERAS 63,Manuel Caldeiro,Paseo de la Castellana nº 298
245,Colegio Público Vázquez de Mella,Colegios públicos,CALLE BAILEN 18,Plaza de San Francisco,Plaza de San Francisco nº 5
246,Colegio Público Winston Churchill,Colegios públicos,AVENIDA GRAN VIA DEL ESTE 5,Pavones,Calle Hacienda de Pavones nº 342
247,Colegio Público de Prácticas Asunción Rincón,Colegios públicos,AVENIDA FILIPINAS 5,Parque Santander,Calle Santander esquina con Avenida de Filipinas


## Store the data

In [ ]:
# Store the results in a new '.csv' file
result_df.to_csv("../data/result/result.csv", index=False)

In [17]:
def show_one_school(df, station_type, school_name):
    # Short dataframe and extract the columns interesting to the goal
    #df = short_store_data(df, station_type)

    # Create a filter with the rows that includes the specific lab
    filter_df = df['Place of interest'] == school_name
    # Evaluate if at least one element in condition is True. If True, it means that there is at least one row that meets the condition. If not
    # the return is a error message
    if filter_df.any():
        return df[filter_df]
    else:
        return 'Error: the name of the lab you typed was not found'

In [18]:
test = show_one_school(result_df, 'bicimad', 'Colegio Público Adolfo Suárez')
test

,Place of interest,Type of place,Place address,BiciMAD station,Station location
0,Colegio Público Adolfo Suárez,Colegios públicos,CALLE INFANTA CATALINA MICAELA 31,Manuel Caldeiro,Paseo de la Castellana nº 298


## Bonus 1

In [ ]:
# This bonus was done in '.py'

## Bonus 2

In [22]:
def find_nearest_bicimad(df, name_school):
    # With this function it is possible to get the best match for 'school_name' in the different school names
    best_match = process.extractOne(name_school, result_df['Place of interest'])
    best_match = process.extractOne(school_name, df['Place of interest'])
    print(best_match)

    # If the coincidence is higher than 80%, show the BiciMAD station
    if best_match[1] >= 80:  
        # Extract the row of the chosen school
        choice_school = df.loc[result_df['Place of interest'] == best_match[0]]
        # Extract the value bicimad station name
        bicimad_nearest = choice_school['BiciMAD station'].values
        # Show the result
        return f"The nearest BiciMAD station to the school {name_school} is {bicimad_nearest}."
    else:
        return "No close match was found for the school name."
    
result_str = find_nearest_bicimad(result_df, 'adolfo suarez')
result_str

('Colegio Público Adolfo Suárez', 86, 0)


"The nearest BiciMAD station to the school adolfo suarez is ['Manuel Caldeiro']."

In [51]:
df = result_df
best_match = process.extractOne('adolfo suarez', df['Place of interest'])
if best_match[1] >= 80:
    filter_df = df['Place of interest'] == best_match[0]
    result_df = df[filter_df]

result_all = result_df
result_df['BiciMAD station'][0]

filter_station = bicimad_df['name'] == result_df['BiciMAD station'][0]
result_station_df =bicimad_df[filter_station]
result_station_df.head()

# Create list with only the interesting columns
interesting_columns = ['total_bases', 'dock_bikes', 'free_bases']
# Create list with the new names to rename the columns
new_columns_names = ['Total bases', 'Dock bikes', 'Free bases']

# Extract the free bases from the dataframe and include that with result_df
data_to_insert = list(result_station_df[interesting_columns].values)
print(data_to_insert)
# Store this data in the dataframe with the other information
result_all[interesting_columns] = data_to_insert

result_all

[array([24, 11, 11], dtype=int64)]


,Place of interest,Type of place,Place address,BiciMAD station,Station location,total_bases,dock_bikes,free_bases
0,Colegio Público Adolfo Suárez,Colegios públicos,CALLE INFANTA CATALINA MICAELA 31,Manuel Caldeiro,Paseo de la Castellana nº 298,24,11,11


## Bonus 3

In [4]:
config = dotenv_values('.env')
client_id = config('CLIENT_ID')   # CLIENT ID
client_secret = config['CLIENT_SECRET']   # CLIENT SECRET

TypeError: 'collections.OrderedDict' object is not callable

In [ ]:
#https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/
base_url = "https://openapi.emtmadrid.es/"
endpoint_login = f"v1/mobilitylabs/{client_id}/login/"
full_endpoint = BASE_URL + ENDPOINT_LOGIN
#related_artists_response = requests.get(full_endpoint, headers = header_info)   #Get to spotify


In [ ]:
https://openapi.emtmadrid.es/v1/transport/bicimadgo/bikes/availability/